In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import pymc as pm
sns.set()


In [ ]:
dick = pd.read_pickle('country_data.pkl')



In [ ]:


def process_country(country):
    df = dick[country]
    
    mask = df['confirmed'] > 1000
    df = df[mask]
    mask = df['deceased'] > 5
    df = df[mask]
    
    confirmed_mean = df['confirmed'].mean()
    confirmed_std = df['confirmed'].std()
    dead_mean = df['deceased'].mean()
    dead_std = df['deceased'].std()
    
    #confirmed_s = (df['confirmed'] - confirmed_mean) / confirmed_std
    #dead_s = (df['deceased'] - dead_mean) / dead_std
    
    confirmed_s = df['confirmed']
    dead_s = df['deceased']
    
    plt.plot(confirmed_s,dead_s,'x--')
    
    M_mu = pm.Uniform('M_mu',lower=0.1,upper=1)
    M_sigma = pm.Uniform('M_sigma',lower=0,upper=1)
    F_mu = pm.Uniform('F_mu',lower=1,upper=5)
    F_sigma = pm.Uniform('F_sigma',lower=0,upper=0.1)
    
    M = pm.Lognormal('M',mu=M_mu,tau=1/M_sigma ** 2)
    F = pm.Lognormal('F',mu=F_mu,tau=1/F_sigma ** 2)
    
    
    @pm.deterministic()
    def gen_dead(confirmed_s=confirmed_s,F=F,M=M):
        return confirmed_s * F * M
    
    obs = pm.Poisson('obs',mu=gen_dead,observed=True,value=dead_s)
    
    model = pm.Model([M_mu,M_sigma,F_mu,F_sigma,M,F,gen_dead,obs])
    map_ = pm.MAP(model)
    #map_.fit()
    mcmc = pm.MCMC(model)
    mcmc.sample(1000000,500000,2)
    
    M_mu_post = mcmc.trace('M_mu')[:]
    M_sigma_post = mcmc.trace('M_sigma')[:]
    F_mu_post = mcmc.trace('F_mu')[:]
    F_sigma_post = mcmc.trace('F_sigma')[:]
    M_post = mcmc.trace('M')[:]
    F_post = mcmc.trace('F')[:]
    
    result = pd.DataFrame({'M_mu_post' : M_mu_post,
                          'M_sigma_post' : M_sigma_post,
                          'F_mu_post' : F_mu_post,
                          'F_sigma_post' : F_sigma_post,
                          'M_post' : M_post,
                          'F_post' : F_post})
    
    return result

def plot(country_name,result):
    plt.figure(figsize=(18,12))
    plt.subplot(211)
    plt.title('Bayesian estimation of Mortality rate for {}'.format(country_name))
    plt.xlabel('Mortality rate')
    plt.ylabel('Density')
    plt.hist(result.M_post,density=True,bins=10)

    plt.subplot(212)
    plt.title('Bayesian estimation of "Of-By-Factor for Confirmed" rate for {}'.format(country_name))
    plt.xlabel('Factor for Confirmed')
    plt.ylabel('Density')
    plt.hist(result.F_post,density=True,bins=10)

    plt.savefig('Bayesian_estimate_mortality_and_factor_{}_.jpg'.format(country_name),format='jpg')

country_list = ['Cruise Ship','Iran']

for c in country_list:
    result = process_country(c)
    plot(c,result)
    print (result.describe())
